mul_aa = (MULT a a)
mul_bb = (MULT b b)
mul_cc = (MULT c c)
PYTHAGOREAN = λa.λb.λc (EQ (PLUS mul_aa, mul_bb) mul_cc)

In [1]:
import time

from lambda_calculus import Variable as Var
from lambda_calculus import Abstraction as Lambda
from lambda_calculus import Application as App

from lambda_calculus.visitors.normalisation import BetaNormalisingVisitor

from lambda_calculus.terms import logic
from lambda_calculus.terms import arithmetic
from lambda_calculus.terms import pairs
from lambda_calculus.terms import combinators

In [2]:
def multi_app_term(term_0, term_1, *terms):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term

In [ ]:
def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def mult_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(m_, App(plus_term(), n_), n_term(0))))

In [3]:
def leq_term():
    m, n = Var("mmm"), Var("nnn")
    return Lambda("mmm", Lambda("nnn", App(
        arithmetic.ISZERO,
        App(App(arithmetic.SUBTRACT, m), n)
    )))


def eq_term():
    mm, nn = Var("mm"), Var("nn")
    return Lambda("mm", Lambda("nn", multi_app_term(
        logic.AND,
        multi_app_term(leq_term(), mm, nn),
        multi_app_term(leq_term(), nn, mm)
    )))

In [7]:
def pythagorean_term():
    a, b, c = "a", "b", "c"
    a_, b_, c_ = Var(a), Var(b), Var(c)

    mul_aa = multi_app_term(arithmetic.MULTIPLY, a_, a_)
    mul_bb = multi_app_term(arithmetic.MULTIPLY, b_, b_)
    mul_cc = multi_app_term(arithmetic.MULTIPLY, c_, c_)

    plus_aa_bb = multi_app_term(arithmetic.ADD, mul_aa, mul_bb)

    return Lambda(a, Lambda(b, Lambda(c, multi_app_term(eq_term(), plus_aa_bb, mul_cc))))


def test(a=0, b=0, c=0):
    term_ = multi_app_term(pythagorean_term(), arithmetic.number(a), arithmetic.number(b), arithmetic.number(c))
    t_start = time.time()
    term_res = BetaNormalisingVisitor().skip_intermediate(term_)
    total_time = time.time() - t_start

    print(f"Time: {total_time}")
    print(term_res)

In [8]:
test(0, 0, 0)

Time: 0.002000570297241211
(λx.(λy.x))


In [9]:
test(1, 0, 1)

Time: 0.002000570297241211
(λx.(λy.x))


In [10]:
test(0, 1, 1)

Time: 0.0020008087158203125
(λx.(λy.x))


In [11]:
test(3, 4, 5)

Time: 0.22805070877075195
(λx.(λy.x))


In [12]:
test(2, 2, 4)

Time: 0.023004531860351562
(λx.(λy.y))


In [13]:
test(3, 0, 3)

Time: 0.013003110885620117
(λx.(λy.x))


In [14]:
test(16, 0, 16)

KeyboardInterrupt: 